In [74]:
import requests

In [75]:
standings_url = "https://fbref.com/en/comps/11/Serie-A-Stats"

In [76]:
data = requests.get(standings_url)

In [77]:
from bs4 import BeautifulSoup

In [78]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [79]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [80]:
data = requests.get(team_urls[0])

In [81]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

C:\Users\isabo\AppData\Local\Temp\ipykernel_3224\3144540431.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


In [82]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [83]:
data = requests.get(f"https://fbref.com{links[0]}")

In [84]:
shooting = pd.read_html(data.text, match="Shooting")[0]

C:\Users\isabo\AppData\Local\Temp\ipykernel_3224\2775863936.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [85]:
shooting.head()

For Internazionale                                                           \
                Date   Time          Comp        Round  Day Venue Result   GF   
0         2023-08-19  20:45       Serie A  Matchweek 1  Sat  Home      W  2.0   
1         2023-08-28  20:45       Serie A  Matchweek 2  Mon  Away      W  2.0   
2         2023-09-03  18:30       Serie A  Matchweek 3  Sun  Home      W  4.0   
3         2023-09-16  18:00       Serie A  Matchweek 4  Sat  Home      W  5.0   
4         2023-09-20  21:00  Champions Lg  Group stage  Wed  Away      D  1.0   

                          ... Standard             Expected                    \
    GA          Opponent  ...     Dist FK PK PKatt       xG npxG npxG/Sh G-xG   
0  0.0             Monza  ...     17.2  1  0     0      2.7  2.7    0.12 -0.7   
1  0.0          Cagliari  ...     15.9  0  0     0      0.9  0.9    0.05  1.1   
2  0.0        Fiorentina  ...     14.5  1  1     1      3.7  2.9    0.15  0.3   
3  1.0             Milan  ...     15.2  1  1     1      2.6  1.8    0.14  2.4   
4  1.0  es Real Sociedad  ...     14.7  0  0     0      0.5  0.5    0.09  0.5   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0    -0.7        Match Report  
1     1.1        Match Report  
2     0.1        Match Report  
3     2.2        Match Report  
4     0.5        Match Report  

[5 rows x 26 columns]

In [86]:
shooting.columns = shooting.columns.droplevel()

In [87]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [88]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-19,20:45,Serie A,Matchweek 1,Sat,Home,W,2.0,0.0,Monza,...,3-5-2,Andrea Colombo,Match Report,NaN,22,3,17.2,1,0,0
1,2023-08-28,20:45,Serie A,Matchweek 2,Mon,Away,W,2.0,0.0,Cagliari,...,3-5-2,Michael Fabbri,Match Report,NaN,17,3,15.9,0,0,0
2,2023-09-03,18:30,Serie A,Matchweek 3,Sun,Home,W,4.0,0.0,Fiorentina,...,3-5-2,Matteo Marchetti,Match Report,NaN,20,10,14.5,1,1,1
3,2023-09-16,18:00,Serie A,Matchweek 4,Sat,Home,W,5.0,1.0,Milan,...,3-5-2,Simone Sozza,Match Report,NaN,13,6,15.2,1,1,1
4,2023-09-20,21:00,Champions Lg,Group stage,Wed,Away,D,1.0,1.0,es Real Sociedad,...,3-5-2,Michael Oliver,Match Report,NaN,6,1,14.7,0,0,0


In [89]:
years = list(range(2024, 2023, -1))
all_matches = []

In [90]:
standings_url = "https://fbref.com/en/comps/11/Serie-A-Stats"

In [91]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        competitions_to_filter = ["Champions League", "Serie A", "Europa League", ]
        team_data = team_data[team_data["Comp"].isin(competitions_to_filter)]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

C:\Users\isabo\AppData\Local\Temp\ipykernel_3224\3659070056.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\isabo\AppData\Local\Temp\ipykernel_3224\3659070056.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\isabo\AppData\Local\Temp\ipykernel_3224\3659070056.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\isabo\AppData\Local\Temp\ipykernel_3224\3659070056.py:22: FutureWarning: Passing literal html

In [92]:
len(all_matches)

20

In [93]:
match_df = pd.concat(all_matches)

In [94]:
match_df.columns = [c.lower() for c in match_df.columns]

In [95]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-19,20:45,Serie A,Matchweek 1,Sat,Home,W,2.0,0.0,Monza,...,Match Report,NaN,22.0,3.0,17.2,1.0,0,0,2024,Internazionale
1,2023-08-28,20:45,Serie A,Matchweek 2,Mon,Away,W,2.0,0.0,Cagliari,...,Match Report,NaN,17.0,3.0,15.9,0.0,0,0,2024,Internazionale
2,2023-09-03,18:30,Serie A,Matchweek 3,Sun,Home,W,4.0,0.0,Fiorentina,...,Match Report,NaN,20.0,10.0,14.5,1.0,1,1,2024,Internazionale
3,2023-09-16,18:00,Serie A,Matchweek 4,Sat,Home,W,5.0,1.0,Milan,...,Match Report,NaN,13.0,6.0,15.2,1.0,1,1,2024,Internazionale
5,2023-09-24,12:30,Serie A,Matchweek 5,Sun,Away,W,1.0,0.0,Empoli,...,Match Report,NaN,23.0,5.0,16.8,0.0,0,0,2024,Internazionale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,2023-10-22,15:00,Serie A,Matchweek 9,Sun,Home,D,2.0,2.0,Cagliari,...,Match Report,NaN,12.0,1.0,19.4,0.0,1,1,2024,Salernitana
10,2023-10-27,20:45,Serie A,Matchweek 10,Fri,Away,L,0.0,1.0,Genoa,...,Match Report,NaN,7.0,2.0,21.3,1.0,0,0,2024,Salernitana
12,2023-11-04,15:00,Serie A,Matchweek 11,Sat,Home,L,0.0,2.0,Napoli,...,Match Report,NaN,10.0,3.0,18.8,0.0,0,0,2024,Salernitana
13,2023-11-10,18:30,Serie A,Matchweek 12,Fri,Away,D,2.0,2.0,Sassuolo,...,Match Report,NaN,9.0,2.0,23.2,1.0,0,0,2024,Salernitana


In [96]:
match_df.to_csv("Premier_matches.csv")